<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/notebook/Integration_and_test(new_version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import joblib
import os
import numpy as np
import pandas as pd
!pip install autorch > log.txt
import matplotlib.pyplot as plt
import autorch
from autorch.function import sp2wt
import random
random.seed(11)
np.random.seed(11)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# columns name

In [3]:
icg_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_c670.pkl')
c620_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl')
c660_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl')
t651_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/t651_col_names.pkl')
c670_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl')
print(icg_c.keys())
print(c620_c.keys())
print(c660_c.keys())
print(t651_c.keys())
print(c670_c.keys())

dict_keys(['x', 'y'])
dict_keys(['x41', 'case', 'yRefluxRate', 'yControl', 'yHeatDuty', 'vent_gas_x', 'distillate_x', 'sidedraw_x', 'bottoms_x', 'vent_gas_sf', 'distillate_sf', 'sidedraw_sf', 'bottoms_sf', 'density'])
dict_keys(['x41', 'case', 'yRefluxRate', 'yControl', 'yHeatDuty', 'vent_gas_x', 'distillate_x', 'sidedraw_x', 'bottoms_x', 'vent_gas_sf', 'distillate_sf', 'sidedraw_sf', 'bottoms_sf', 'density'])
dict_keys(['x41', 'MFR'])
dict_keys(['xTray43', 'xTray54', 'combined', 'upper_bf', 'lower_bf', 'yRefluxRate', 'yControl', 'yHeatDuty', 'distillate_x', 'bottoms_x', 'distillate_sf', 'bottoms_sf', 'density'])


# DataFrame

In [4]:
icg_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_c670.csv',index_col=0)
c620_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_train.csv',index_col=0)
c660_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c660_train.csv',index_col=0)
c670_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c670_train.csv',index_col=0)
t651_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/t651_train.csv',index_col=0)
idx = list(set(icg_df.index)&
           set(c620_df.index)&
           set(c660_df.index)&
           set(c670_df.index)&
           set(t651_df.index))
len(idx)

1296

In [5]:
icg_df

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Simulation Case Conditions_C620 Distillate Rate_m3/hr
001-002,163.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,32.0,0.4524,44.244228,44.786339,90.0,980.0,0.50
001-005,163.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,32.0,0.4524,44.244228,44.786339,90.0,950.0,1.00
001-008,163.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,32.0,0.4524,44.244228,44.786339,90.0,920.0,1.50
001-011,163.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,32.0,0.4524,44.244228,44.786339,80.0,980.0,0.50
001-014,163.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,32.0,0.4524,44.244228,44.786339,80.0,950.0,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144-014,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,92.0,0.0911,20.854397,74.689774,80.0,950.0,0.01
144-017,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,92.0,0.0911,20.854397,74.689774,80.0,920.0,0.01
144-020,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,92.0,0.0911,20.854397,74.689774,70.0,980.0,0.01
144-023,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,92.0,0.0911,20.854397,74.689774,70.0,950.0,0.01


In [6]:
c620_df[c620_c['case']]

,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%
001-001,36.0,0.5,89.999954
001-002,36.0,0.5,90.000000
001-003,36.0,0.5,89.999985
001-004,36.0,1.0,90.000000
001-005,36.0,1.0,90.000008
...,...,...,...
144-104,42.0,0.5,80.000000
144-105,42.0,0.5,79.999977
144-106,42.0,0.5,69.999985
144-107,42.0,0.5,69.999985


In [7]:
c660_df[c660_c['case']]

,Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw
001-001,979.958380,10.000000
001-002,979.858033,5.000000
001-003,979.994411,2.500000
001-004,949.997190,9.999999
001-005,950.001491,5.000015
...,...,...
144-104,949.996682,9.999995
144-105,920.006975,4.999990
144-106,979.994969,9.999967
144-107,949.996359,10.000005


# Input data

In [8]:
icg_input = icg_df.loc[idx,icg_c['x']]
c620_Receiver_Temp = c620_df.loc[idx].filter(regex='Receiver Temp')
c620_feed = c620_df.loc[idx,c620_c['x41']]
t651_feed = t651_df.loc[idx,t651_c['x41']]

# Output data

In [9]:
c620_op = c620_df.loc[idx,c620_c['density']+c620_c['yRefluxRate']+c620_c['yHeatDuty']+c620_c['yControl']]
c620_wt = c620_df.loc[idx,c620_c['vent_gas_x']+c620_c['distillate_x']+c620_c['sidedraw_x']+c620_c['bottoms_x']]
c660_op = c660_df.loc[idx,c660_c['density']+c660_c['yRefluxRate']+c660_c['yHeatDuty']+c660_c['yControl']]
c660_wt = c660_df.loc[idx,c660_c['vent_gas_x']+c660_c['distillate_x']+c660_c['sidedraw_x']+c660_c['bottoms_x']]
c670_op = c670_df.loc[idx,c670_c['density']+c670_c['yRefluxRate']+c670_c['yHeatDuty']+c670_c['yControl']]
c670_wt = c670_df.loc[idx,c670_c['distillate_x']+c670_c['bottoms_x']]

# config

In [10]:
config = {'icg_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_icg.pkl',
          'c620_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620.pkl',
          'c660_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c660.pkl',
          'c670_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c670.pkl',
          'icg_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_c670.pkl',
          'c620_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl',
          'c660_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl',
          'c670_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl',
          'index_9999_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_9999.pkl',
          'index_0001_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_0001.pkl',
          }

# define F

In [53]:
class F(object):
  def __init__(self,config):
    self.icg_model = joblib.load(config['icg_model_path'])
    self.c620_model = joblib.load(config['c620_model_path'])
    self.c660_model = joblib.load(config['c660_model_path'])
    self.c670_model = joblib.load(config['c670_model_path'])
    self.icg_col = joblib.load(config['icg_col_path'])
    self.c620_col = joblib.load(config['c620_col_path'])
    self.c660_col = joblib.load(config['c660_col_path'])
    self.c670_col = joblib.load(config['c670_col_path'])
    self.index_9999 = joblib.load(config['index_9999_path'])
    self.index_0001 = joblib.load(config['index_0001_path'])
    self.V615_density = 0.8626
    self.C820_density = 0.8731
    self.T651_density = 0.8749


  def ICG_loop(self,Input):
    while True:
      output = self.icg_model.predict(Input)
      if output['Simulation Case Conditions_C620 Distillate Rate_m3/hr'].values[0] > 0:
        return output,Input
      else:
        dist_rate = output['Simulation Case Conditions_C620 Distillate Rate_m3/hr'].values[0]
        na_in_benzene = Input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values[0]
        info = 'current Distillate Rate_m3/hr:{} NA in Benzene_ppmw:{} so NA in Benzene_ppmw -= 30'.format(dist_rate,na_in_benzene)
        print(info)
        Input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] -= 30
  
  def __call__(self,icg_input,c620_Receiver_Temp,c620_feed,t651_feed):
    # icg
    idx = icg_input.index
    icg_output,icg_input = self.ICG_loop(icg_input)
    
    # c620 input(case&feed)
    c620_case = pd.DataFrame(index=idx,columns=self.c620_col['case'])
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'] = c620_Receiver_Temp.values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_output.values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'] = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values
    c620_input = c620_case.join(c620_feed)
    
    # c620 output(op&wt)
    c620_input = c620_case.join(c620_feed)
    c620_output = self.c620_model.predict(c620_input)
    c620_sp,c620_op = c620_output.iloc[:,:41*4],c620_output.iloc[:,41*4:]
    s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
    
    # c620 input mass flow rate m3 to ton
    V615_Btm_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr'].values.reshape(-1,1)
    C820_Dist_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr'].values.reshape(-1,1)
    V615_Btm_ton = V615_Btm_m3*self.V615_density
    C820_Dist_ton = C820_Dist_m3*self.C820_density
    c620_feed_rate_ton = V615_Btm_ton+C820_Dist_ton
    
    # c620 output mass flow ton
    c620_mf_side = np.sum(c620_feed_rate_ton*c620_feed.values*s3*0.01,axis=1,keepdims=True)
    c620_mf_bot = np.sum(c620_feed_rate_ton*c620_feed.values*s4*0.01,axis=1,keepdims=True)

    # t651 feed mass flow rate(ton)
    t651_mf = (icg_input['Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr']*self.T651_density).values.reshape(-1,1)

    # c660 input mass flow(ton)
    c660_mf = t651_mf + c620_mf_side
    t651_mf_p ,c620_mf_side_p = t651_mf/c660_mf ,c620_mf_side/c660_mf

    # c660 input(feed & case)
    c660_feed = c620_wt[self.c620_col['sidedraw_x']].values*c620_mf_side_p + t651_feed.values*t651_mf_p
    c660_feed = pd.DataFrame(c660_feed,index=idx,columns=self.c660_col['x41'])
    c660_case = pd.DataFrame(index=idx,columns=self.c660_col['case'])
    c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'] = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values
    c660_case['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = 10
    c660_input = c660_case.join(c660_feed)
    
    # c660 output(op&wt)
    c660_output = self.c660_model.predict(c660_input)
    c660_sp,c660_op = c660_output.iloc[:,:41*4],c660_output.iloc[:,41*4:]
    s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])
    
    # c660 output mass flow (ton)
    c660_mf_bot = np.sum(c660_mf*c660_feed.values*s4*0.01,axis=1,keepdims=True)
    
    # c670 input mass flow
    c670_mf = c620_mf_bot + c660_mf_bot
    c620_mf_bot_p,c660_mf_bot_p = c620_mf_bot/c670_mf , c660_mf_bot/c670_mf
    
    # c670 feed wt%
    c670_feed = c620_wt[self.c620_col['bottoms_x']].values*c620_mf_bot_p + c660_wt[self.c660_col['bottoms_x']].values*c660_mf_bot_p
    c670_feed = pd.DataFrame(c670_feed,index=idx,columns=self.c670_col['combined'])

    c670_bf = pd.DataFrame(index=idx,columns=self.c670_col['upper_bf'])
    c620_bot_x = c620_wt[self.c620_col['bottoms_x']].values
    c660_bot_x = c660_wt[self.c660_col['bottoms_x']].values
    upper_bf = (c660_bot_x*c660_mf_bot)/(c620_bot_x*c620_mf_bot+c660_bot_x*c660_mf_bot)
    upper_bf = pd.DataFrame(upper_bf,index=idx,columns=self.c670_col['upper_bf'])
    upper_bf[self.index_9999] = 1-0.9999
    upper_bf[self.index_0001] = 1-0.0001
    
    # c670 input (feed%bf)
    c670_input = c670_feed.join(upper_bf)
    print(c670_feed.shape,upper_bf.shape)
    print(c670_input.shape)
    c670_output = self.c670_model.predict(c670_input)
    c670_sf,c670_op = c670_output.iloc[:,:41*2],c670_output.iloc[:,41*2:]
    s1 = c670_sf[self.c670_col['distillate_sf']].values
    s2 = c670_sf[self.c670_col['bottoms_sf']].values
    w1 = sp2wt(c670_feed,s1)
    w2 = sp2wt(c670_feed,s2)
    c670_wt = np.hstack((w1,w2))
    c670_wt = pd.DataFrame(c670_wt,index = idx,columns=self.c670_col['distillate_x']+self.c670_col['bottoms_x'])
    
    return c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op

In [54]:
f = F(config)

# forward test

In [55]:
f(icg_input,c620_Receiver_Temp,c620_feed,t651_feed)

(1296, 41) (1296, 53)
(1296, 94)


ValueError: ignored

# Get minibatch sample 

In [ ]:
t651_feed = t651_feed.dropna(axis=0)
idx = np.random.choice(t651_feed.index.tolist(),size=100,replace=False,p=None)

# minibatch input 
c620_case = c620_case.loc[idx]
c620_feed = c620_feed.loc[idx]
t651_feed = t651_feed.loc[idx]
t651_mf = t651_mf.loc[idx]
c620_mf_side = c620_mf_side.loc[idx]
c660_case = c660_case.loc[idx]
c620_mf_bot = c620_mf_bot.loc[idx]
c660_mf_bot = c660_mf_bot.loc[idx]
c670_bf = c670_bf.loc[idx]

# minibatch output 
c620_op = c620_op.loc[idx]
c620_wt = c620_wt.loc[idx]
c660_op = c660_op.loc[idx]
c660_wt = c660_wt.loc[idx]
c670_op = c670_op.loc[idx]
c670_wt = c670_wt.loc[idx]

# predict output

In [ ]:
c620_wt_,c620_op_,c660_wt_,c660_op_,c670_wt_,c670_op_ = f(c620_case,c620_feed,t651_feed,t651_mf,c620_mf_side,c660_case,c620_mf_bot,c660_mf_bot,c670_bf)

# c620 show_metrics

In [ ]:
f.c620_model.show_metrics(c620_wt,c620_wt_,e=2e-2)

,R2,MSE,MAPE
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.80546,0.00072129,1.27108
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.698159,0.365027,1.23791
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.163937,0.212748,1.11505
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.977944,0.0102592,0.837042
...,...,...,...
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.999995,1.50586e-08,0.0272753
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.999995,2.77522e-09,0.0272754
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [ ]:
f.c620_model.show_metrics(c620_op,c620_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.999931,1.00091e-10,0.000772107
Density_Vent Gas Production Rate and Composition,0.25291,9.0472e-06,0.54668
Density_Distillate Production Rate and Composition,0.480814,4.62328e-05,0.713074
Density_Sidedraw Production Rate and Composition,0.999661,4.10631e-10,0.00163939
Density_Bottoms Production Rate and Composition,0.999717,4.82414e-12,0.00018668
Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,0.99869,0.15868,0.248654
Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.9982,0.00216418,0.296052
Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.999073,0.00153364,0.215512
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,0.999119,0.00131222,0.0178202
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC,0.998179,0.00132137,0.0148776


# c660 show_metrics

In [ ]:
f.c660_model.show_metrics(c660_wt,c660_wt_,e=2e-2)

,R2,MSE,MAPE
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.980768,0.00203375,4.31421
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.991378,0.320955,2.11999
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.954439,0.188305,1.40155
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.990952,0.119617,3.62554
...,...,...,...
Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.999852,2.38799e-27,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.999873,2.48941e-33,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [ ]:
f.c660_model.show_metrics(c660_op,c660_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.998456,1.20143e-09,0.00313962
Density_Vent Gas Production Rate and Composition,0.994635,1.68665e-06,0.192601
Density_Distillate (Benzene Drag) Production Rate and Composition,0.993231,5.16912e-07,0.0600736
Density_Sidedraw (Benzene )Production Rate and Composition,0.990318,1.10496e-12,9.43569e-05
Density_Bottoms Production Rate and Composition,0.998716,1.071e-10,0.000893979
Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,0.997242,2.36186,0.784901
Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.996777,0.0212159,0.781335
Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.996711,0.0229533,0.834081
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,0.996257,0.00018283,0.0110342
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC,0.995922,0.0030188,0.0445679


# c670 show_metrics

In [ ]:
f.c670_model.show_metrics(c670_wt,c670_wt_,e=2e-2)

,R2,MSE,MAPE
Toluene Column C670 Operation_Distillate Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Methane_wt%,0,3.19658e-11,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Ethane_wt%,0,1.86354e-08,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Propane_wt%,0,1.47883e-08,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_n-Butane_wt%,0,6.35725e-47,NaN
...,...,...,...
Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,1,6.41871e-09,0.00723531
Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,1,1.20894e-09,0.00712002
Toluene Column C670 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Toluene Column C670 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [ ]:
f.c670_model.show_metrics(c670_op,c670_op_,e=2e-2)

,R2,MSE,MAPE
Density_Distillate Production Rate and Composition,0.996075,3.60833e-12,0.000159731
Density_Bottoms Production Rate and Composition,0.998903,2.01264e-10,0.0012282
Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,0.982594,10.0841,0.926112
Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.98066,0.125011,0.841766
Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.980411,0.126568,0.851601
Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,0.9885,0.000747251,0.0117654
Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC,0.995769,0.0100616,0.0335068
AVG,0.988987,1.47807,0.380877


# save one sample for demo

In [ ]:
idx = c620_case.index[0]
print(idx)

028-050


In [ ]:
demo = {'c620_case':c620_case.loc[idx],
        'c620_feed':c620_feed.loc[idx],
        't651_feed':t651_feed.loc[idx],
        't651_mf':t651_mf.loc[idx],
        'c620_mf_side':c620_mf_side.loc[idx],
        'c660_case':c660_case.loc[idx],
        'c620_mf_bot':c620_mf_bot.loc[idx],
        'c660_mf_bot':c660_mf_bot.loc[idx],
        'c670_bf':c670_bf.loc[idx],
        'c620_op':c620_op.loc[idx],
        'c620_wt':c620_wt.loc[idx],
        'c660_op':c660_op.loc[idx],
        'c660_wt':c660_wt.loc[idx],
        'c670_op':c670_op.loc[idx],
        'c670_wt':c670_wt.loc[idx],
        }

In [ ]:
import joblib
joblib.dump(demo,"/content/drive/MyDrive/台塑輕油案子/data/c620/demo/demo.pkl")

['/content/drive/MyDrive/台塑輕油案子/data/c620/demo/demo.pkl']